In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
x = pd.read_csv("C:\\Users\\User\\Downloads\\train.csv", index_col ='Id')
xy_test = pd.read_csv("C:\\Users\\User\\Downloads\\test.csv",index_col='Id')

In [3]:
print(x.head())
print(xy_test.head())

    MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                    
1           60       RL         65.0     8450   Pave   NaN      Reg   
2           20       RL         80.0     9600   Pave   NaN      Reg   
3           60       RL         68.0    11250   Pave   NaN      IR1   
4           70       RL         60.0     9550   Pave   NaN      IR1   
5           60       RL         84.0    14260   Pave   NaN      IR1   

   LandContour Utilities LotConfig  ... PoolArea PoolQC Fence MiscFeature  \
Id                                  ...                                     
1          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
2          Lvl    AllPub       FR2  ...        0    NaN   NaN         NaN   
3          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
4          Lvl    AllPub    Corner  ...        0    NaN   NaN         NaN   
5          Lvl    AllPub       FR2  ... 

In [4]:
x.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [5]:
#removing null value

x.dropna(axis=0,subset=['SalePrice'],inplace = True)

y = x.SalePrice
x.drop(['SalePrice'],inplace = True,axis=1)

In [6]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [7]:
cat_col = [cname for cname in x_train.columns if
          x_train[cname].nunique()<10 and
          x_train[cname].dtype == 'object']

In [8]:
num_col = [cname for cname in x_train.columns if 
            x_train[cname].dtype in ['int64', 'float64']]

In [11]:
# Keep selected columns only
my_cols = cat_col + num_col
X_train = x_train [my_cols].copy()
X_valid = x_test[my_cols].copy()
X_test = xy_test[my_cols].copy()

In [12]:
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
136,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,530,98,0,0,0,0,0,0,5,2008
1453,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,525,0,28,0,0,0,0,0,5,2006
763,FV,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,614,169,45,0,0,0,0,0,6,2010
933,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,RRNn,Norm,...,788,0,191,0,0,0,0,0,3,2007
436,RL,Pave,NaN,IR2,Lvl,AllPub,CulDSac,Gtl,PosN,Norm,...,550,158,61,0,0,0,0,0,4,2009


In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_col),
        ('cat', categorical_transformer, cat_col)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_test, preds))

MAE: 17267.74105022831


# Step 1: Improve the performance

In [16]:
#preprocessing numerical data
num_transformer = SimpleImputer(strategy = 'constant')
#preprocessing categorical data

cat_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

In [18]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_col),
        ('cat', cat_transformer, cat_col)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

In [19]:
my_pipline = Pipeline(steps=[('preprocessor',preprocessor),
                            ('model',model)]).fit(X_train,y_train)

In [20]:
pred = my_pipline.predict(X_valid)

In [21]:
score = mean_absolute_error(y_test,pred)
print('MAE',score)


MAE 17230.161643835618


In [24]:
pred_test = my_pipline.predict(X_test)
pred_test

array([126225.  , 153163.5 , 183936.84, ..., 153374.  , 113284.  ,
       227443.53])

In [26]:
from sklearn.model_selection import cross_val_score

scores = -1 * cross_val_score(my_pipline,x,y, cv=5,
                             scoring="neg_mean_absolute_error")

print("Average MAE score",score.mean())

Average MAE score 17230.161643835618
